In [1]:
import os
import pyNetLogo
import matplotlib.pyplot as plt
import pandas as pd
import ipyparallel
import os
import pickle
import copy
netlogo_home = "/netlogo-5.3.1-64"
netlogo_version = "5"
#os.chdir("C:\\Users\\DDawso01\\OneDrive - Environmental Protection Agency (EPA)\\Profile\\Documents\\Bee Project\\Beehave_PEEM_Nectar")
os.chdir("/home/ddawso01/Documents/BeeProject/BeeHave_PEEM_Nectar_DED")
netlogo_home = "/netlogo-5.3.1-64"
netlogo_version = "5"
Suff="DED_070220"
params=14

ll=0.5
ul=2

probparams = {
'num_vars': 14,
'names': [
'NectarPesticideInput',
'AdultAcutePar1',
'AdultAcutePar2',
'LarvaAcutePar1',
'LarvaAcutePar2',
'CROPVOLUME',
'LIFESPAN',
'MAX_BROOD_NURSE_RATIO',
'MAX_EGG_LAYING',
'MAX_PROPORPORTION_POLLEN_FORAGERS',
'MORTALITY_FOR_PER_SEC',
'MORTALITY_INHIVE',
'POLLENLOAD',
'TIME_NECTAR_GATHERING'],
'bounds': [
[0, 200], #above and below the experiment concentrations
[0.0001,0.02 ], #concentrations below what you get with 10ppb and above 200 ppb
[-10,-1], #wide ranges for this parameer
[0.0001,0.02], #concentrations below what you get with 10ppb and above 200 ppb
[-10,-1], #wide ranges for this parameter
[50*ll, 50*ul], #Cropvolume
[290*ll, 290*ul], #Lifespan
[3*ll, 3*ul], #Max_brood_Nurse_Ratio
[1600*ll, 1600*ul], #Max_Egg_laying
[0.8*ll, 0.8*ul],#Max_prop_pollen_foragers
[0.00001*ll, 0.00001*ul], #Mortality_for_per_sec
[0.004*ll, 0.004*ul], #Mortality_inhive                  
[0.015*ll, 0.015*ul], #POllenload
[1200*ll, 1200*ul] #Time_Nectar_Gathering; time spent 
                    


]
}
#Note; need to find parameter ranges for new parameters. 
from SALib.sample import saltelli
from SALib.analyze import sobol
n=1000
param_values=saltelli.sample(probparams, n, calc_second_order=True)


In [ ]:
#Specification of Constants and/or variable things not of interest, including weather, food, start day, random-seed and runlength
import numpy as np
#Note: files are looked for in the directory of the jupyter notebook instead of the directory specified below via chdir. Thats peculiar; hopefully it won't be a problem when it is converted to a .py file 
#For some reason, having trouble specifying text files directly, will have to deal with later 
#weathervec=np.repeat("\"Weather_WI_08_2016_input.txt\"",  param_values.shape[0])
#foodinputvec=np.repeat("\"EnvFile_1flowerpatch_fromPython.txt\"",  param_values.shape[0])
startday=np.repeat(171, param_values.shape[0])
runlength=np.repeat(124, param_values.shape[0])
randomseed = np.random.uniform(0,10000,param_values.shape[0])
paramindex = list(range(1,(param_values.shape[0]+1))) #Have to put list around range to produce the range of numbers; don't need list inside a loop becuase its an iterator
param_values=np.column_stack((param_values, startday,runlength, randomseed, paramindex))

#Note: np.vstack can be used similarly to add a row

In [2]:

# Store data (serialize)
#with open('SobolParameterTab.pickle', 'wb') as handle:
#    pickle.dump(param_values, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
with open('SobolParameterTab.pickle', 'rb') as handle:
    param_values = pickle.load(handle)

#print(your_data == unserialized_data)

In [3]:
param_values.shape

(3000, 18)

In [4]:
#Creating Final Problem Set with Input File constants
#problem = {
#'num_vars': 7,
#'names': [
#'NectarPesticideInput',
#'NectarSugarConcentrationInput',
#'NectarVolumeInput', 'Input_File', 'WeatherFile', 'StartDay', 'X_Days'
#]}
problem = {
'num_vars': 18,
'names': [
'NectarPesticideInput',
'AdultAcutePar1',
'AdultAcutePar2',
'LarvaAcutePar1',
'LarvaAcutePar2' ,
'CROPVOLUME',
'LIFESPAN',
'MAX_BROOD_NURSE_RATIO',
'MAX_EGG_LAYING',
'MAX_PROPORTION_POLLEN_FORAGERS',
'MORTALITY_FOR_PER_SEC',
'MORTALITY_INHIVE',
'POLLENLOAD',
'TIME_NECTAR_GATHERING',
 'StartDay', 'X_Days', 'RAND_SEED', 'paramindex'
]}





In [ ]:

SenParamLabel="SobalSA_paramsets" + str(params) + "_params_" + str(n) + "reps_" +  str(((params * 2) + 2) * n ) + "sims_" +  Suff + ".csv"

pv.to_csv(SenParamLabel)


In [122]:
def simulation(experiment):
    #Because the model clears everything with setup, setup is called with default values first, and then filled in with variable values
    #prior to starting model runs 
    #Need to stipulate that experiment= TRUE and startdayinput from the git go
    #Set the input parameters
        #for i, name in enumerate(problem['names']):
    #    if name == 'random-seed':
     #       #The NetLogo random seed requires a different syntax
      #      netlogo.command('random-seed {}'.format(experiment[i]))
        
      
     #   if name == 'paramindex':
      #      pass
       
    
       # elif name == 'paramindex':
        #    pass
        
       # else:
        #    #Otherwise, assume the input parameters are global variables
         #   netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')

    #A second parameter setting sesssion is issued to set the parameter values after a "clear-all" command clears
    #parameter values set on the "Code" page
    for i, name in enumerate(problem['names']):
    #    if name == 'random-seed':
     #       #The NetLogo random seed requires a different syntax
      #      netlogo.command('random-seed {}'.format(experiment[i]))
        
      
        if name == 'paramindex':
            pass
       
    
       # elif name == 'paramindex':
        #    pass
        
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))
 


    #Run for the specified number of ticks and return the values of the variables specified at each timestep
    #Note that this outputs a dataframe, which is transferred into a list per output with parallel processing
    counts = netlogo.repeat_report(['NectarPesticideInput','TotalEggs', 'TotalLarvae', 'TotalPupae', 'TotalIHBees', 
                                    'TotalForagers', 'AFF', 'NewWorkerEggs', 'HoneyEnergyStore', 
                                    'HoneyStorePesticideConc', 'PollenBugAlarm'], 124)


    return counts
    

In [120]:
client = ipyparallel.Client()
direct_view = client[:]
direct_view

<DirectView [0]>

In [7]:
%%px
import os 
import pyNetLogo
import pandas as pd
netlogo_home = "/home/ddawso01/Documents/BeeProject/netlogo-5.3.1-64"
netlogo_version = "5"
netlogo=pyNetLogo.NetLogoLink(gui=False, netlogo_home=netlogo_home, netlogo_version=netlogo_version)
netlogo.load_model("/home/ddawso01/Documents/BeeProject/BeeHave_PEEM_Nectar_DED/Beehave_PEEM_Nectar_Sensitivity_DED063020.nlogo")

#import pyNetLogo
#import pandas as pd
#netlogo_home = "C:\\Program Files\\NetLogo 5.3.1"
#netlogo_version = "5"
#netlogo=pyNetLogo.NetLogoLink(gui=False, netlogo_home=netlogo_home, netlogo_version=netlogo_version)
#netlogo.load_model('C:\\Users\\DDawso01\\OneDrive - Environmental Protection Agency (EPA)\\Profile\\Documents\\Bee Project\\Beehave_PEEM_Nectar\\Beehave_PEEM_Nectar_Sensitivity_DED063020.nlogo')




In [8]:
import os
cwd=os.getcwd()
os.chdir(cwd)
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))
direct_view.push(dict(problem=problem))

<AsyncResult: _push>

In [123]:
#%%time
#Note, have to run a new lview for every run; otherwise it won't work. 

#had to add this in because I forgot that the range 0:10 excludes 10; so had to fill in with other simulations
lview = client.load_balanced_view()
results = lview.map_sync(simulation, param_values)
with open('SenRun_07022020.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)


#Bug alarm: Pollen consumption results in increased pollen stores; this has to do with some issue where all the workers die really early on 
#I'm guessing that pollen gets added without anybody eating it, messing up the accounting; not really a problem here, since the colony fails rapidly.  

In [124]:
resultstest

[      NectarPesticideInput TotalEggs TotalLarvae TotalPupae TotalIHBees  \
 171.0              43.9453         0           0          0         347   
 172.0              43.9453      5345       10943      14270        8054   
 173.0              43.9453      4675       11509      14799        9157   
 174.0              43.9453      4077       12018      15318       10269   
 175.0              43.9453      5329       10688      15808       11369   
 ...                    ...       ...         ...        ...         ...   
 290.0              43.9453        27          71        248         381   
 291.0              43.9453        24          65        232         407   
 292.0              43.9453        22          62        214         435   
 293.0              43.9453        22          56        200         459   
 294.0              43.9453        22          51        187         482   
 
       TotalForagers AFF NewWorkerEggs HoneyEnergyStore  \
 171.0          7600  21   

In [31]:
    
with open('SenRun_07022020.pickle', 'rb') as handle:
    results=pickle.load(handle)


In [87]:
results

,TotalEggs,TotalLarvae,TotalPupae,TotalIHBees,TotalForagers,AFF,NewWorkerEggs,HoneyEnergyStore,HoneyStorePesticideConc,PollenBugAlarm
171.0,0,0,0,463,10100,21,0,1.23527e+06,0,0
172.0,0,289,19090,10780,11000,21,204,1.22866e+06,0,0
173.0,1066,288,17511,12285,10900,22,1066,1.22195e+06,0,0
174.0,2228,285,15941,13786,10900,23,1188,1.21509e+06,0,0
175.0,3476,279,14366,15282,10900,24,1311,1.20807e+06,0,0
...,...,...,...,...,...,...,...,...,...,...
290.0,26,63,124,56,5554,20,8,1.278e+06,0.0024336,0
291.0,24,9,124,38,5436,21,8,1.27718e+06,0.0024336,0
292.0,22,16,124,40,5336,21,7,1.27639e+06,0.0024336,0
293.0,21,24,124,35,5336,21,7,1.2756e+06,0.0024336,0


In [55]:
#Loop over elements of the list and create a large dataframe#Note, this takes a long time(like 20 min)
#import numpy as np
#resultdf=results[0]
#resultdf['paramindex'] = np.repeat(1, resultdf.shape[0])
#for i in range(1, (len(results))):
#    resultdf1=results[i]
#    resultdf1['paramindex'] = np.repeat((i + 1), resultdf1.shape[0])
#    resultdf=resultdf.append(resultdf1) # can be used similarly to add a row

#This does it directly 
resultdf=pd.concat(results)



In [102]:
#Set Day as a columsn
resultdf.index.name = 'Day'
resultdf.reset_index(inplace=True)

In [117]:
resultdf[(resultdf["Day"]==323)]
#This is an option to control the display of dataframes
#pd.set_option('display.max_rows', 1000)
#results100[79]
#param_values[78]

,Day,TotalEggs,TotalLarvae,TotalPupae,TotalIHBees,TotalForagers,AFF,NewWorkerEggs,HoneyEnergyStore,HoneyStorePesticideConc,PollenBugAlarm,paramindex
123,323.0,2,7,31,105,2200,15,1,1.25581e+06,0.0019352,0,1
247,323.0,2,2,12,5,1592,13,1,1.27754e+06,0.00114925,0,2
371,323.0,2,8,27,129,4270,14,1,1.25548e+06,0.00153401,0,3
495,323.0,3,5,24,81,1200,14,1,1.27309e+06,0.00188405,0,4
124123,323.0,3,12,37,210,11187,17,1,1.20428e+06,0.0054016,0,1001
124247,323.0,3,11,35,186,6651,15,1,1.22946e+06,0.00611051,0,1002
124371,323.0,3,4,16,9,6293,13,1,1.27622e+06,0.00574109,0,1003
124495,323.0,3,10,41,211,6179,16,1,1.23826e+06,0.0041317,0,1004
248123,323.0,4,11,36,96,2815,13,1,1.26152e+06,0.00509546,0,2001
248247,323.0,3,12,34,166,1595,17,1,1.2658e+06,0.00608444,0,2002


In [118]:
param_values[:4]

array([[ 4.39453125e+01,  2.02392578e-03, -5.33300781e+00,
         1.35674805e-02, -7.47753906e+00,  9.30419922e+01,
         1.64965820e+02,  5.54736328e+00,  2.00234375e+03,
         4.83203125e-01,  6.27441406e-06,  3.52929687e-03,
         1.11254883e-02,  1.29082031e+03,  1.71000000e+02,
         1.24000000e+02,  7.97228326e+03,  1.00000000e+00],
       [ 2.87109375e+01,  2.02392578e-03, -5.33300781e+00,
         1.35674805e-02, -7.47753906e+00,  9.30419922e+01,
         1.64965820e+02,  5.54736328e+00,  2.00234375e+03,
         4.83203125e-01,  6.27441406e-06,  3.52929687e-03,
         1.11254883e-02,  1.29082031e+03,  1.71000000e+02,
         1.24000000e+02,  6.03837333e+03,  2.00000000e+00],
       [ 4.39453125e+01,  7.46533203e-03, -5.33300781e+00,
         1.35674805e-02, -7.47753906e+00,  9.30419922e+01,
         1.64965820e+02,  5.54736328e+00,  2.00234375e+03,
         4.83203125e-01,  6.27441406e-06,  3.52929687e-03,
         1.11254883e-02,  1.29082031e+03,  1.71000000e

In [60]:
#SenResults="SobalSenResults_1year_3params_1000reps_8000_Parameterizations_" + Suff + ".csv"
#SenResults="SobalSA_Sims_" + str(params) + "params_" + str(n) + "reps_" + str((2^3) * n) + "pmts_" +  Suff + ".csv"
SenResults="SobalSA_Sims" + str(params) + "_params_" + str(n) + "reps_" +  str(((params * 2) + 2) * n ) + "sims_" +  Suff + ".csv"
resultdf.to_csv(SenResults)

In [ ]:
import os
import pyNetLogo
import matplotlib.pyplot as plt
import pandas as pd
#os.getcwd()
resultdf=pd.read_csv("SobalSA_Sims14_params_100reps_3000sims_DED_06242020.csv")

In [62]:
#This is run again, in case we're doing this separate from the above.
import os
import pyNetLogo
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas import Grouper
from SALib.sample import saltelli
from SALib.analyze import sobol
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import scipy
#os.chdir('C:\\Users\\DDawso01\\OneDrive - Environmental Protection Agency (EPA)\\Profile\\Documents\\Bee Project\\Beehave_PEEM_Nectar\\SobalSims_010920')
#Loading parameter values and simulations
params=pd.read_csv("SobalSA_paramsets14_params_100reps_3000sims_DED_06242020.csv")
sims=pd.read_csv("SobalSA_Sims14_params_100reps_3000sims_DED_06242020.csv")

In [73]:
print(params.columns)
print(sims.columns)

#Data is stored per day on a parameter index basis. Want to do a couple of things
#1. Want to make some plots of activity of the whole year for the different output parameters
#2. Want to decide on some summary statistics to do a sensitivity analyses on, including paramter effects on:

#TotalEggs(this is influenced by ELR_t, which is responsive to the number of nurse bees available, given the option is turned on in BeeHave, which it was for these simulations
#TotalForagers
#TotalIHBees
#HoneyEnergyStore
#HoneyStorePesticideConc
#Day of colony collapse(as indicated by) TotalForagers+TotalIHBees=0





Index(['Unnamed: 0', 'NectarPesticideInput', 'AdultAcutePar1',
       'AdultAcutePar2', 'LarvaAcutePar1', 'LarvaAcutePar2', 'CROPVOLUME',
       'LIFESPAN', 'MAX_BROOD_NURSE_RATIO', 'MAX_EGG_LAYING',
       'MAX_PROPORTION_POLLEN_FORAGERS', 'MORTALITY_FOR_PER_SEC',
       'MORTALITY_INHIVE', 'POLLENLOAD', 'TIME_NECTAR_GATHERING', 'StartDay',
       'X_Days', 'random_seed', 'paramindex'],
      dtype='object')
Index(['Unnamed: 0', 'TotalEggs', 'TotalLarvae', 'TotalPupae', 'TotalIHBees',
       'TotalForagers', 'AFF', 'NewWorkerEggs', 'HoneyEnergyStore',
       'HoneyStorePesticideConc', 'PollenBugAlarm', 'paramindex'],
      dtype='object')


In [92]:
#re-name unnamed to day
sims=sims.rename(columns={'Unnamed: 0':  'Day'})

In [75]:
#Drop unncessary additional column
params.drop(['Unnamed: 0'], axis=1)

,NectarPesticideInput,AdultAcutePar1,AdultAcutePar2,LarvaAcutePar1,LarvaAcutePar2,CROPVOLUME,LIFESPAN,MAX_BROOD_NURSE_RATIO,MAX_EGG_LAYING,MAX_PROPORTION_POLLEN_FORAGERS,MORTALITY_FOR_PER_SEC,MORTALITY_INHIVE,POLLENLOAD,TIME_NECTAR_GATHERING,StartDay,X_Days,random_seed,paramindex
0,43.945312,0.002024,-5.333008,0.013567,-7.477539,93.041992,164.965820,5.547363,2002.343750,0.483203,0.000006,0.003529,0.011125,1290.820312,171.0,124.0,7972.283261,1.0
1,28.710938,0.002024,-5.333008,0.013567,-7.477539,93.041992,164.965820,5.547363,2002.343750,0.483203,0.000006,0.003529,0.011125,1290.820312,171.0,124.0,6038.373331,2.0
2,43.945312,0.007465,-5.333008,0.013567,-7.477539,93.041992,164.965820,5.547363,2002.343750,0.483203,0.000006,0.003529,0.011125,1290.820312,171.0,124.0,9524.067244,3.0
3,43.945312,0.002024,-3.522461,0.013567,-7.477539,93.041992,164.965820,5.547363,2002.343750,0.483203,0.000006,0.003529,0.011125,1290.820312,171.0,124.0,6575.803456,4.0
4,43.945312,0.002024,-5.333008,0.006960,-7.477539,93.041992,164.965820,5.547363,2002.343750,0.483203,0.000006,0.003529,0.011125,1290.820312,171.0,124.0,3163.938033,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,99.902344,0.008291,-4.062988,0.008855,-6.488770,42.175293,471.887207,2.016357,1225.390625,1.500977,0.000017,0.006491,0.011422,2237.402344,171.0,124.0,2041.244282,2996.0
2996,99.902344,0.008291,-4.062988,0.008855,-6.488770,42.175293,471.887207,2.016357,1225.390625,1.500977,0.000016,0.005044,0.011422,2237.402344,171.0,124.0,7394.533288,2997.0
2997,99.902344,0.008291,-4.062988,0.008855,-6.488770,42.175293,471.887207,2.016357,1225.390625,1.500977,0.000016,0.006491,0.028143,2237.402344,171.0,124.0,8343.103881,2998.0
2998,99.902344,0.008291,-4.062988,0.008855,-6.488770,42.175293,471.887207,2.016357,1225.390625,1.500977,0.000016,0.006491,0.011422,2179.394531,171.0,124.0,7410.152808,2999.0


In [95]:
subset=sims[sims["Day"]==200]

In [99]:
len(param_values)

3000

In [ ]:
import time
exprange=list(range(1,8001))
datalist=[]
start = time.time()
for slot in exprange:
    pset=sims[sims["paramindex"] == slot]
    a=pset["TotalEggs"].max()
    b=pset["TotalEggs"].mean()
    c=pset["TotalEggs"].median()
    d=pset["TotalEggs"].min()
    e=pset["TotalForagers"].max()
    f=pset["TotalForagers"].mean()
    g=pset["TotalForagers"].median()
    h=pset["TotalForagers"].min()
    i=pset["TotalIHBees"].max()
    j=pset["TotalIHBees"].mean()
    k=pset["TotalIHBees"].median()
    l=pset["TotalIHBees"].min()
    m=pset["HoneyEnergyStore"].max()
    n=pset["HoneyEnergyStore"].mean()
    o=pset["HoneyEnergyStore"].median()
    p=pset["HoneyEnergyStore"].min()
    q=pset["HoneyStorePesticideConc"].max()
    r=pset["HoneyStorePesticideConc"].median()
    s=pset["HoneyStorePesticideConc"].mean()   
    Adults=pd.DataFrame(pset["TotalForagers"] + pset["TotalIHBees"])
    Adults.columns=["TotalAdults"]
    Adults["Days"]=list(range(1,366))
    
    if Adults["TotalAdults"].min() == 0:
        Adults=Adults[Adults["TotalAdults"] == 0]  
        t= Adults.iloc[0,1]
    
    else: t= 365
    u=pset["AFF"].mean()
    v=pset["TotalLarvae"].mean()
    w=pset["TotalLarvae"].median()
    x=slot
    datalist.extend((a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x))
elapsed_time_fl = (time.time() - start) 
    #Need to collate all this stuff into a list; maybe like concatenate or something? 
print(elapsed_time_fl)


In [ ]:
pset["AFF"].plot(kind="hist")

In [ ]:
datalistarray=np.array(datalist)
datalistdf=pd.DataFrame(datalistarray.reshape(-1,24)) #here is the first mention of datalistdf
datalistdf.columns=["Eggs_Max","Eggs_Mean", "Eggs_Median", "Eggs_Min", 
                    "Foragers_Max", "Foragers_Mean", "Foragers_Median", "Foragers_Min",
                    "IHBees_Max", "IHBees_Mean", "IHBees_Median", "IHBees_Min",
                    "EnergyStore_Max", "EnergyStore_Mean", "EnergyStore_Median", "EnergyStore_Min",
                    "PestConc_Max", "PestConc_Median", "PestConc_Mean",                    
                    "ColonyCollapseDay","AFF_Mean","Larvae_Mean", "Larvae_Median","paramindex"]
datalistdf.to_csv("SobalSimsOutput_DED01092020.csv")


In [ ]:
#This produces a grid of plots showing the distributions of the output variables 
fig = plt.figure(figsize=(25,15))
fig.subplots_adjust(hspace=1.0, wspace=0.3)
for i in range(1,(datalistdf.shape[1])):
    ax=fig.add_subplot(5,5,i)
    plt.hist(datalistdf.iloc[:,i-1])
    plt.xlabel(datalistdf.columns[i-1])
plt.savefig("SenSims_Distributions.png", dpi=100)


In [ ]:
#Individual or multiple plots
#pset=sims[sims["paramindex"] == 2]
#print(pset["TotalEggs"].max())
#plt.plot(list(range(1,366)),pset["TotalEggs"])

for i in list(range(1,1000)):
    pset=sims[sims["paramindex"] == i]
    plt.plot(list(range(1,366)),pset["TotalForagers"])

In [ ]:
#Merge datalistdf with params
#datalistdf.merge(self, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
dltotal=datalistdf.merge(params, left_on='paramindex', right_on='paramindex')
dltotal



In [ ]:
#Create Function to create plots
def pairwiseplots(paramset,dataset,variable,ylimadjust, ncol=4,nrow=1):
    fig, ax = plt.subplots(nrow, ncol, sharey=True)
    y = dataset[variable]
    for i, a in enumerate(ax.flatten()):
        x = paramset.iloc[:,i]
        sns.regplot(x, y, ax=a, ci=None, color='k',scatter_kws={'alpha':0.2, 's':4, 'color':'gray'})
        spearman = scipy.stats.spearmanr(x, y)
        a.annotate("r: {:6.3f}".format(spearman[0]), xy=(0.15, 0.85), xycoords='axes fraction',fontsize=13)
        if divmod(i,ncol)[1]>0:
            a.get_yaxis().set_visible(False)
        a.set_xlabel(paramset.columns[i])
        a.set_ylim([0,ylimadjust*np.max(y)])

    fig.set_size_inches(14,4,forward=True)
    fig.subplots_adjust(wspace=0.1, hspace=0.3)
    return plt.show();


In [ ]:
params1=params.iloc[:,[1,2,3,6]]


In [ ]:
params.columns

In [ ]:
pairwiseplots(paramset=params1, dataset=datalistdf, variable="PestConc_Mean",ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="PestConc_Max",ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="Eggs_Max",ylimadjust=0.8)
pairwiseplots(paramset=params1, dataset=datalistdf, variable="Eggs_Median",ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="Eggs_Median",ylimadjust=0.8))
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="Foragers_Max", ylimadjust=0.8)
pairwiseplots(paramset=params1, dataset=datalistdf, variable="Foragers_Mean", ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="Foragers_Median",ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="Foragers_Min", ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="IHBees_Max", ylimadjust=0.8)
pairwiseplots(paramset=params1, dataset=datalistdf, variable="IHBees_Median", ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="IHBees_Median",ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="IHBees_Min", ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="EnergyStore_Max", ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="EnergyStore_Median", ylimadjust=0.8)
pairwiseplots(paramset=params1, dataset=datalistdf, variable="EnergyStore_Median", ylimadjust=0.8)
#pairwiseplots(paramset=params1, dataset=datalistdf, variable="EnergyStore_Min", ylimadjust=0.8)
pairwiseplots(paramset=params1, dataset=datalistdf, variable="ColonyCollapseDay", ylimadjust=1.1)
pairwiseplots(paramset=params1, dataset=datalistdf, variable="AFF_Mean", ylimadjust=1.1)
pairwiseplots(paramset=params1, dataset=datalistdf, variable="Larvae_Mean", ylimadjust=1.1)


It looks like pesticide concentration input impact concentration in the honey, and negatively impacts demographics. Interestingly, all of the impacts to demographics appear to be in a negative logistic way; that is demographic values rapidly decline with increases in pesticide concentration, and then plateau. The reverse patter occurs for the impact of pesticides on AFF. I think this is because as pesticide starts killing off foragers and IHbees, there are fewer individuals around to care for the brood. So, AFF goes up. Then, there are fewer nurse bees so egglaying rate goes down. However, there is a limit on AFF, so eventually the system evens out. Sugar concentration has strong logistic-growth influence on colony collapse day and a linear relationship on energy stores. There also appears to have a minimum threshold (about 0.5 mol/l) wherein demographics increase before spreading out. Quantity of nectar doesn't seem to influence anything. Looking a the model, this appears to be because anything above 1 liter (or 1000000 microliters) seems to allow the colony to survive until the end of the year. So, for sensitity analyses purpurses, we'd probably need to reduce liters to less than 1 to get a better picture. Interestingly, there  


In [ ]:
probparams = {
'num_vars': 3,
'names': [
'NectarPesticideInput',
'NectarSugarConcentrationInput',
'NectarVolumeInput' 
],
}



In [ ]:
#Note, the sobal analyze function assumes that the parameter set used to generate the datavalues was generated using a sobal saltelli sampler, with
#values ordered in a particular way. Thus, it doesn't need the actual parameter values, just the problemset in same order as 
#the parameter set was generated. 
def sobolfunction (Variable, dataset, problemset):
    Si = sobol.analyze(problemset, dataset[Variable].values, calc_second_order=True, print_to_console=False) 
    Si_filter = {k:Si[k] for k in ['ST','ST_conf','S1','S1_conf']}
    Si_df = pd.DataFrame(Si_filter, index=problemset['names'])
    fig, ax = plt.subplots(1)

    indices = Si_df[['S1','ST']]
    err = Si_df[['S1_conf','ST_conf']]

    indices.plot.bar(yerr=err.values.T,ax=ax, title=Variable)
    
    fig.set_size_inches(8,4)
    
    #return print(Si_df)
    return plt.show()

In [ ]:
sobalfunction("Eggs_Mean", datalistdf, probparams)
sobalfunction("AFF_Mean", datalistdf, probparams)
sobalfunction("Foragers_Mean", datalistdf, probparams)
sobalfunction("IHBees_Mean", datalistdf, probparams)

In [ ]:
Si_df

It appears that the model is running like it should, as far as I can tell. Pesticide impacts have interesting influences on demographics and sugar concentration. It sounds like for the purposes of a reasonable field study in which sugar availability and concentration are not an issue, anything above 1 mol/l of sugar concentration (like 1.5 to be safe), and maybe above 1 liter per day (2 to be safe) should produce steady population dynamics in the absense of pesticides. So, to get an age distribution of bees, I'll do a bunch of runs (like 1000) with  those paramter settings but with varied random-seeds, and up to June 1st (a day in the middle of summer). Then, I'll take the average age distribution in each stage that I can use to initialize simulations from. I can probably do this as a function in the NetLogo model. 

After that is set up, I'll need to try to figure out how to use the ABCpy function. 